In [16]:
import numpy as np

w = np.array([0.5, 0.5])

def obj_value(w):
    # A = np.random.randn(2, 2)
    # Q = np.dot(A.T, A)
    b = np.array([1, 1])
    obj_value = 0.5 * w.T @ w - np.dot(b, w)
    grad = w - b
    return obj_value, grad

gradient_check = np.zeros_like(w)
h = 1e-11
for i in range(len(w)):
    w_forward = np.copy(w)
    w_backward = np.copy(w)
    w_forward[i] += h
    print("w = ", w)
    print("w_forward = ", w_forward)
    print(obj_value(w_forward)[0])
    print(obj_value(w)[0])
    # w_backward[i] -= h
    gradient_check[i] = (obj_value(w_forward)[0] - obj_value(w_backward)[0]) / h
    print(gradient_check[i])

grad = obj_value(w)[1]

print(gradient_check - grad)

w =  [0.5 0.5]
w_forward =  [0.5 0.5]
-0.750000000005
-0.75
-0.5000000413701855
w =  [0.5 0.5]
w_forward =  [0.5 0.5]
-0.750000000005
-0.75
-0.5000000413701855
[-4.13701855e-08 -4.13701855e-08]


In [31]:


def obj_value(w):
        x = np.array([1, 2])
        tilde_BG = np.array([[1, 2], [3, 4]]) * 100
        Bv = np.array([1, 1])
        l_lower = 0.2
        l_upper = 1
        theta = 10
        m = tilde_BG.shape[1]
        obj_value = (
            - np.dot((tilde_BG.T @ x + Bv), w)
            + np.linalg.norm(tilde_BG @ w) ** 2 / (2 * (l_upper - l_lower))
            - (np.sum(np.log(w))) / theta
        )
        # obj_value = (
        #     - np.dot((tilde_BG.T @ x + Bv), w)
        #     + np.linalg.norm(tilde_BG @ w) ** 2 / (2 * (l_upper - l_lower))
        #     + (np.log(m) + np.dot(w, np.log(w))) / theta
        # )
        return obj_value
def obj_gradient(w):
        x = np.array([1, 2])
        tilde_BG = np.array([[1, 2], [3, 4]]) * 100
        Bv = np.array([1, 1])
        l_lower = 0.2
        l_upper = 1
        theta = 10
        m = tilde_BG.shape[1]
        obj_gradient = ((-1/w) / theta 
                    + tilde_BG.T @ tilde_BG @ w / (l_upper - l_lower) 
                    - tilde_BG.T @ x - Bv
        )
        # obj_gradient = (np.log(w) / theta 
        #             + np.ones(m) / theta 
        #             + tilde_BG.T @ tilde_BG @ w / (l_upper - l_lower) 
        #             - tilde_BG.T @ x - Bv
        # )
        # check gradient
        gradient_check = np.zeros_like(w)
        h = 1e-8
        for i in range(len(w)):
            w_forward = np.copy(w)
            w_backward = np.copy(w)
            w_forward[i] += h
            # w_backward[i] -= h
            gradient_check[i] = (obj_value(w_forward) - obj_value(w_backward)) / h
            print(gradient_check[i])

        print("check: ", gradient_check - obj_gradient)
        return obj_gradient

w = np.array([0.5, 0.5])
print(obj_gradient(w))

149298.79980627447
211498.79903532565
check:  [-0.00019373 -0.00096467]
[149298.8 211498.8]
